# **Aquisição de dados com raspagem de dados**

In [1]:
# Instalação de pacotes para raspagem e análise de dados.
!pip install beautifulsoup4 pandas requests

In [2]:
# Declara define funções utilitárias utilizadas no notebook.
import datetime
def formata_msg(nivel, msg):
    """
    Formata uma mensagem de log incluindo o nível de severidade, timestamp
    e a mensagem.

    Parâmetros:
    - nivel (str): Nível de severidade da mensagem (ex: 'INFO', 'ERROR', 'WARNING').
    - msg (str): A mensagem de log propriamente dita.

    Retorna:
    - str: A mensagem de log formatada.
    """
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    return f"[{nivel}] {timestamp} - {msg}"

In [3]:
# Importa módulos essenciais para funcionalidades do notebook.
import os

import pandas as pd
import requests
from bs4 import BeautifulSoup

print(formata_msg("INFO", "Bibliotecas importadas com sucesso; ambiente pronto."))

[INFO] 2024-05-01 22:33:01 - Bibliotecas importadas com sucesso; ambiente pronto.


In [4]:
# Define o diretório local para armazenar as bases de dados públicas coletadas.
corpora_caminho = "./corpora"

# Verifica se o diretório especificado já existe no ambiente do Colab.
if not os.path.exists(corpora_caminho):
  # Caso o diretório não exista, cria um novo diretório com o nome 'corpora'.
  # Isso é útil para organizar os arquivos de dados baixados ou gerados.
  print(formata_msg("INFO", f"Diretório {corpora_caminho} foi criado com sucesso."))
  os.mkdir(corpora_caminho)
else:
  print(formata_msg("INFO", f"Diretório {corpora_caminho} já existe no sistema."))

[INFO] 2024-05-01 22:33:01 - Diretório ./corpora foi criado com sucesso.


## 1. G1

## Raspagem de dados de notícias do G1

O G1 é um portal de notícias brasileiro mantido pelo Grupo Globo, sob a orientação da Central Globo de Jornalismo. Lançado em 18 de setembro de 2006, no ano em que a TV Globo completou 41 anos, o portal reúne conteúdos jornalísticos das diversas empresas do Grupo Globo — incluindo TV Globo, GloboNews, rádio CBN, os jornais O Globo, Extra, Valor Econômico, a revista Globo Rural, entre outros — além de produzir reportagens exclusivas em formatos de texto, fotos, áudio e vídeo.

Os elementos estruturais da página principal, essenciais para a raspagem de dados, incluem as seguintes classes CSS:

* **feed-post-body:** corpo da notícia;
* **feed-post-link:** título da notícia;
* **feed-post-body-resumo:** subtítulo da notícia (opcional).



In [5]:
# Declara a URL do site para a raspagem de dados.
url = "https://g1.globo.com/"

In [6]:
# Define o nome do arquivo onde os dados serão salvos.
noticias_arquivo = "g1_noticias.csv"

# Usa 'os.path.join' para criar um caminho completo, garantindo compatibilidade
# entre sistemas operacionais.
noticias_caminho = os.path.join(corpora_caminho, noticias_arquivo)

# Inicializa uma lista para armazenar os dados das notícias.
noticias_g1 = []

# Faz uma solicitação HTTP GET para a URL especificada.
resposta = requests.get(url)

# Verifica se a solicitação foi bem-sucedida (código 200).
if resposta.status_code == 200:
  # Processa o conteúdo da página se a solicitação for bem-sucedida.
  pagina = resposta.content
  soup = BeautifulSoup(pagina, 'html.parser')

  # Encontra todos os elementos que correspondem ao corpo da notícia.
  noticias = soup.findAll('div', attrs={'class': 'feed-post-body'})

  # Itera sobre cada notícia encontrada.
  for noticia in noticias:
    # Extrai o título da notícia.
    titulo = noticia.find('a', attrs={'class': 'feed-post-link'})

    # Tenta encontrar o subtítulo da notícia.
    subtitulo = noticia.find('div', attrs={'class': 'feed-post-body-resumo'})

    # Adiciona o título, subtítulo (se houver) e o link à lista de notícias.
    if (subtitulo):
      noticias_g1.append([titulo.text, subtitulo.text, titulo['href']])
    else:
      noticias_g1.append([titulo.text, '', titulo['href']])

  # Converte a lista de notícias em um DataFrame para facilitar a manipulação.
  noticias_g1_df = pd.DataFrame(noticias_g1, columns=['titulo', 'subtitulo', 'link'])

  # Salva os dados coletados no arquivo CSV especificado.
  noticias_g1_df.to_csv(noticias_caminho, index=False)

  print(formata_msg("INFO", f"Dados coletados com sucesso da URL: {url}."))
  print(formata_msg("INFO", f"Os resultados foram salvos no arquivo: {noticias_arquivo}."))
else:
  print("Falha na solicitação, código de status:", resposta.status_code)



[INFO] 2024-05-01 22:33:19 - Dados coletados com sucesso da URL: https://g1.globo.com/.
[INFO] 2024-05-01 22:33:19 - Os resultados foram salvos no arquivo: g1_noticias.csv.


In [9]:
# Carrega os dados do arquivo CSV especificado no DataFrame 'noticias_g1_df'.
noticias_g1_df = pd.read_csv(noticias_caminho)
noticias_g1_df.head(10)

,titulo,subtitulo,link
0,ASSISTA às últimas notícias sobre os temporais...,NaN,https://g1.globo.com/rs/rio-grande-do-sul/ao-v...
1,Mulher arrastada por rio relata como sobrevive...,NaN,https://g1.globo.com/rs/rio-grande-do-sul/noti...
2,Galvão revela papo de Senna que só ele viu ant...,Lenda da Fórmula 1 morreu há 30 anos após acid...,https://g1.globo.com/podcast/o-assunto/noticia...
3,MP denuncia por 2 crimes sobrinha que levou ho...,NaN,https://g1.globo.com/rj/rio-de-janeiro/noticia...
4,Mães denunciam xenofobia e situação precária n...,Hospital registrou 17 mortes a cada mil nascim...,https://g1.globo.com/rr/roraima/noticia/2024/0...
5,BC dos EUA mantém juros em patamar elevado e v...,NaN,https://g1.globo.com/economia/noticia/2024/05/...
6,"Chuvas no Sul, Madonna no Rio e mais vídeos",NaN,https://g1.globo.com/playlist/videos-para-assi...
7,Madonna compartilha memes e foto do palco; 'Hi...,NaN,https://g1.globo.com/rj/rio-de-janeiro/show-da...
